In [1]:
%load_ext autoreload
%autoreload 2

# Plan

In [2]:
import syft as sy
from syft.core.node.common.action.get_object_action import GetObjectAction
from syft.core.node.common.action.function_or_constructor_action import RunFunctionOrConstructorAction
from syft.core.node.common.action.common import Action
from syft.proto.core.node.common.plan.plan_pb2 import Plan as Plan_PB

from google.protobuf.reflection import GeneratedProtocolMessageType

from syft.core.common.uid import UID
from syft.core.io.address import Address
from syft.core.common.serde.deserialize import _deserialize
from syft.decorators.syft_decorator_impl import syft_decorator
from syft.core.common.object import Serializable

In [3]:
from syft.proto.core.node.common.action.action_pb2 import Action as Action_PB
import re

In [4]:
import syft as sy
import torch as th
import sys

In [5]:
from typing import List

In [6]:
CAMEL_TO_SNAKE_PAT = re.compile(r'(?<!^)(?=[A-Z])')


class Plan(Serializable):
    def __init__(self, actions: List[Action]):
        self.actions=actions
        
    @staticmethod
    def get_protobuf_schema() -> GeneratedProtocolMessageType:
        """Return the type of protobuf object which stores a class of this type

        As a part of serialization and deserialization, we need the ability to
        lookup the protobuf object type directly from the object type. This
        static method allows us to do this.

        Importantly, this method is also used to create the reverse lookup ability within
        the metaclass of Serializable. In the metaclass, it calls this method and then
        it takes whatever type is returned from this method and adds an attribute to it
        with the type of this class attached to it. See the MetaSerializable class for details.

        :return: the type of protobuf object which corresponds to this class.
        :rtype: GeneratedProtocolMessageType

        """

        return Plan_PB
    
    @syft_decorator(typechecking=True)
    def _object2proto(self) -> Plan_PB:
        """Returns a protobuf serialization of self.

        As a requirement of all objects which inherit from Serializable,
        this method transforms the current object into the corresponding
        Protobuf object so that it can be further serialized.

        :return: returns a protobuf object
        :rtype: ObjectWithID_PB

        .. note::
            This method is purely an internal method. Please use object.serialize() or one of
            the other public serialization methods if you wish to serialize an
            object.
        """
        def camel_to_snake(s ):
            return CAMEL_TO_SNAKE_PAT.sub('_', s).lower()

        actions_pb = [Action_PB(obj_type = ".".join([action.__module__, action.__class__.__name__]), 
                                **{camel_to_snake(action.__class__.__name__): action.serialize()}
                               )
                      for action in self.actions]
        
        return Plan_PB(actions=actions_pb)

    @staticmethod
    def _proto2object(proto: Plan_PB) -> "GetObjectAction":
        """Creates a ObjectWithID from a protobuf

        As a requirement of all objects which inherit from Serializable,
        this method transforms a protobuf object into an instance of this class.

        :return: returns an instance of GetObjectAction
        :rtype: GetObjectAction

        .. note::
            This method is purely an internal method. Please use syft.deserialize()
            if you wish to deserialize an object.
        """
        actions = []
        
        for action_proto in proto.actions:
            module_parts = action_proto.obj_type.split(".")
            klass = module_parts.pop()
            obj_type = getattr(sys.modules[".".join(module_parts)], klass)
            action_type = action_proto.WhichOneof("action")
            actions.append(obj_type._proto2object(getattr(action_proto, action_type)))

        return Plan(actions=actions)



In [7]:
address=Address()

In [8]:
args = (
    th.tensor([1, 2, 3]),
    th.tensor([4, 5, 5]),
)

In [9]:
a1 = GetObjectAction(id_at_location=UID(), address=address, reply_to=Address(), msg_id=UID())
a2 = RunFunctionOrConstructorAction(path="torch.Tensor.add",args=tuple(),kwargs={},id_at_location=UID(),
                                    address=address,msg_id=UID())

In [10]:
plan = Plan([a1,a2])

In [11]:
blob = plan.serialize()
msg2 = sy.deserialize(blob=blob)

In [12]:
# msg2.actions

# Execution

In [13]:
# alice = sy.VirtualMachine(name="alice")
# alice_client = alice.get_client()

# # args = (
# #     th.tensor([1, 2, 3]).send(alice_client),
# #     th.tensor([4, 5, 5]).send(alice_client),
# # )

# msg = RunFunctionOrConstructorAction(
#     path="torch.Tensor.add",
#     args=(),
#     kwargs={},
#     id_at_location=UID(),
#     address=alice_client.address,
#     msg_id=UID(),
# )

# blob = msg.serialize()

# msg2 = sy.deserialize(blob=blob)

# We first need to add plans and actions to the AST to make them sendable

In [14]:
alice = sy.VirtualMachine(name="alice")
alice_client = alice.get_client()

In [15]:
t = th.tensor([1, 2, 3])

In [16]:
# alice_client.

In [17]:
t.send(alice_client)

In [18]:
action = GetObjectAction(id_at_location=UID(), address=address, reply_to=Address(), msg_id=UID())

In [19]:
from syft.lib import create_lib_ast

In [36]:
alice_client.action.core.node.common.action.get_object_action.GetObjectAction

[2021-02-10T19:29:02.386031+0100][CRITICAL][logger] __getattribute__ failed. If you are trying to access an EnumAttribute or a StaticAttribute, be sure they have been added to the AST. Falling back on__getattr__ to search in self.attrs for the requested field.


[2021-02-10T19:29:02.386215+0100][CRITICAL][logger] 'Class' object has no attribute '_ipython_canary_method_should_not_exist_'
[2021-02-10T19:29:02.386327+0100][CRITICAL][logger] '__getattr__ failed, _ipython_canary_method_should_not_exist_ is not present on the object, nor the AST attributes!'
[2021-02-10T19:29:02.386460+0100][CRITICAL][logger] __getattribute__ failed. If you are trying to access an EnumAttribute or a StaticAttribute, be sure they have been added to the AST. Falling back on__getattr__ to search in self.attrs for the requested field.
[2021-02-10T19:29:02.386562+0100][CRITICAL][logger] 'Class' object has no attribute '_ipython_canary_method_should_not_exist_'
[2021-02-10T19:29:02.386661+0100][CRITICAL][logger] '__getattr__ failed, _ipython_canary_method_should_not_exist_ is not present on the object, nor the AST attributes!'
[2021-02-10T19:29:02.386851+0100][CRITICAL][logger] __getattribute__ failed. If you are trying to access an EnumAttribute or a StaticAttribute, be 

In [37]:
alice_client.action.core.node.common.action.get_object_action.GetObjectAction(id_at_location=UID(), address=address,
                                                                              reply_to=Address(), msg_id=UID())

[2021-02-10T19:32:24.618319+0100][CRITICAL][logger] Path core.node.common.action.get_object_action.GetObjectAction not present in the AST.


ValueError: Path core.node.common.action.get_object_action.GetObjectAction not present in the AST.

In [22]:
# action.send(alice_client)

In [73]:
t_pointer.id_at_location

<UID: bc3477786722490b9ea728e23c5006da>

In [47]:
t.send??